In [ ]:
#Q1
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt 
from keras.datasets import fashion_mnist
(X_train, Y_train), (X_test, Y_test) = fashion_mnist.load_data()
names = ["T-shirt/top","Trouser/pants","Pullover shirt","Dress","Coat","Sandal","Shirt","Sneaker","Bag","Ankle boot"]
dummy = []
for i in range (10):
  for j in range(len(Y_train)):
    if int(Y_train[j]) is i:
      plt.imshow(X_train[j], cmap="gray")
      dummy.append(X_train[j])
      plt.show()
      break 
wandb.log(({"Q1":[wandb.Image(i,caption=j) for i,j in zip(dummy,names)]}))

In [ ]:
def sgd(run,n_epoch,batch_size=100,eta = 0.01):
  global W,b,arguments,n_input,n_points ,n_output, n_hidden_layers,n_hidden_neurons , X,Y,X_t,Y_t,y_o,y_t
  
  d_w , d_b = mul(W , 0) , mul(b,0)
  train_loss , train_acc = [] , []
  val_loss , val_acc = [] , []

  for i in range(n_epoch):
    for j in range(n_points):
      t1 , t2 , _ = gradient(X[j],Y[j],W,b)
      d_w = add(d_w , t1)
      d_b = add(d_b , t2)
      

      if(j%batch_size == 0):
        # print("dw",d_w,"db",d_b)
        W = add(W , d_w , 1, -1*eta)
        b = add(b , d_b , 1, -1*eta)
        d_w , d_b = mul(W , 0) , mul(b,0)
        

    # train acc and loss
    t4 , t5 = accuracy_and_loss(X,Y,y_o)
    # print(W,b)
    train_loss.append(t5)
    train_acc.append(t4)

    # validate
    va , vl = accuracy_and_loss(X_t,Y_t,y_t)
    val_loss.append(vl)
    val_acc.append(va)
    print("epoch",i,"train acc", t4 , "train loss" ,t5 , "validation acc" , va , "validation loss" ,vl)
    # wandb.log({'train_acc' : t4 , 'train_loss' : t5},step = i) 
    vl = float (vl)
    t5 = float (t5)
    run.log({'t_acc' : t4 ,'t_loss' : t5,'v_loss' : vl,'v_acc':va})

  return W , b , train_loss , train_acc , val_loss, val_acc

def momentum_gradient_descent(run,n_epoch,batch_size=100,eta = 0.001,gamma = 0.5):
  
  global W,b,arguments,n_input,n_points ,n_output, n_hidden_layers,n_hidden_neurons , X,Y,X_t,Y_t,y_o,y_t
  
  d_w , d_b = mul(W , 0) , mul(b,0)
  train_loss , train_acc = [] , []
  val_loss , val_acc = [] , []
  p_w , p_b =mul(W , 0) , mul(b,0)
  v_w , v_b =mul(W , 0) , mul(b,0)

  for i in range(n_epoch):
    for j in range(n_points):
      t1 , t2 , _ = gradient(X[j],Y[j],W,b)
      d_w = add(d_w , t1)
      d_b = add(d_b , t2)   
    
      if(j%batch_size == 0):
        v_w , v_b = add(p_w , d_w , gamma , eta) , add(p_b , d_b , gamma , eta)
        W = add(W , v_w , 1, -1)
        b = add(b , v_b , 1 , -1)
        p_w , p_b = v_w , v_b
        # print("dw",d_w,"db",d_b)
        d_w , d_b = mul(W , 0) , mul(b,0)
        
    # train acc and loss
    t4 , t5 = accuracy_and_loss(X,Y,y_o)
    train_loss.append(t5)
    train_acc.append(t4)

    # validate
    va , vl = accuracy_and_loss(X_t,Y_t,y_t)
    val_loss.append(vl)
    val_acc.append(va)
    print("epoch",i,"trian acc", t4 , "train loss" ,t5 , "validation acc" , va , "validation loss" ,vl)
    # wandb.log({'train_acc' : train_acc[-1] , 'train_loss' : train_loss[-1] , 'val_acc' : val_acc[-1], 'val_loss': val_loss[-1]})
    # wandb.log({'train_acc' : t4 , 'train_loss' : t5},step = i)
    vl = float (vl)
    t5 = float (t5)
    run.log({'t_acc' : t4 ,'t_loss' : t5,'v_loss' : vl})
  return W , b , train_loss , train_acc , val_loss, val_acc

In [ ]:
def onehot_encoding(a,n_class):   
  temp = []
  for i in a:
    t1 = np.zeros(n_class)
    t1[i] = 1
    temp.append(t1)
  return temp

def squr(d):    
  temp = {}
  for i in d.keys():
    temp[i] = d[i]**2
  return temp

def mul(d1 , m1 = 1):   
  temp ={}
  for i in d1.keys():
    temp[i] = m1 * d1[i]
  return temp

# dictionary add key wise
def add(d1,d2,m1=1,m2=1):
  temp ={}
  # print(d1,d2,"dd")
  if (m2==0):
    return d1
  for i in d1.keys():
    temp[i] = m1 * d1[i] + m2 * d2[i]
  return temp

In [ ]:
def nesterov_gradient_descent(run,n_epoch,batch_size=100,eta = 0.001,gamma = 0.9):

  global W,b,arguments,n_input,n_points ,n_output, n_hidden_layers,n_hidden_neurons , X,Y,X_t,Y_t,y_o,y_t
  p_v_w , p_v_b = mul(W , 0) , mul(b,0)
  v_w , v_b = mul(W , 0) , mul(b,0)
  d_w , d_b = mul(W , 0) , mul(b,0)
  train_loss , train_acc = [] , []
  val_loss , val_acc = [] , []

  for i in range(n_epoch):
    for j in range(n_points):
      t1 , t2 , _ = gradient(X[j],Y[j],add(W,v_w,1,-gamma),add(b,v_b,1,-gamma))
      d_w = add(d_w , t1)
      d_b = add(d_b , t2)
      
      if(j%batch_size == 0):
        v_w , v_b = add(p_v_w , d_w , gamma , eta) , add(p_v_b , d_b , gamma , eta)
        W = add(W , v_w , 1, -1)
        b = add(b , v_b , 1 , -1)
        d_w , d_b = mul(W , 0) , mul(b,0)
        p_v_w , p_v_b = v_w , v_b

    # train acc and loss
    t4 , t5 = accuracy_and_loss(X,Y,y_o)
    train_loss.append(t5)
    train_acc.append(t4)

    # validate
    va , vl = accuracy_and_loss(X_t,Y_t,y_t)
    val_loss.append(vl)
    val_acc.append(va)
    print("epoch",i,"trian acc", t4 , "train loss" ,t5 , "validation acc" , va , "validation loss" ,vl)
    # wandb.log({'train_acc' : train_acc[-1] , 'train_loss' : train_loss[-1] , 'val_acc' : val_acc[-1], 'val_loss': val_loss[-1]})
    # wandb.log({'train_acc' : t4 , 'train_loss' : t5},step = i)
    vl = float (vl)
    t5 = float (t5)
    run.log({'t_acc' : t4 ,'t_loss' : t5,'v_loss' : vl,'v_acc':va})

  return W , b , train_loss , train_acc , val_loss, val_acc

def adagrad(run,n_epoch,batch_size=100,eta = 0.001,eps = 1e-8):
  
  global W,b,arguments,n_input,n_points ,n_output, n_hidden_layers,n_hidden_neurons , X,Y,X_t,Y_t,y_o,y_t
  v_w , v_b = mul(W , 0) , mul(b,0)
  d_w , d_b = mul(W , 0) , mul(b,0)
  train_loss , train_acc = [] , []
  val_loss , val_acc = [] , []

  for i in range(n_epoch):
    for j in range(n_points):
      t1 , t2 , _ = gradient(X[j],Y[j],W,b)
      d_w = add(d_w , t1)
      d_b = add(d_b , t2)
      
      if(j%batch_size == 0):
        v_w , v_b = add(v_w, squr(d_w)) , add(v_b , squr(d_b))
        W = add(W , adarate(d_w,v_w,eta,eps) , 1, -1)
        b = add(b , adarate(d_b,v_b,eta,eps) , 1 , -1)
        d_w , d_b = mul(W , 0) , mul(b,0)    

    # train acc and loss
    t4 , t5 = accuracy_and_loss(X,Y,y_o)
    train_loss.append(t5)
    train_acc.append(t4)

    # validate
    va , vl = accuracy_and_loss(X_t,Y_t,y_t)
    val_loss.append(vl)
    val_acc.append(va)
    print("epoch",i,"trian acc", t4 , "train loss" ,t5 , "validation acc" , va , "validation loss" ,vl)
    # wandb.log({'train_acc' : train_acc[-1] , 'train_loss' : train_loss[-1] , 'val_acc' : val_acc[-1], 'val_loss': val_loss[-1]})
    # wandb.log({'train_acc' : t4 , 'train_loss' : t5},step = i)
    vl = float (vl)
    t5 = float (t5)
    run.log({'t_acc' : t4 ,'t_loss' : t5,'v_loss' : vl,'v_acc':va})
  return W , b , train_loss , train_acc , val_loss, val_acc 